In [6]:
import pandas as pd
import numpy as np

df = pd.read_pickle('../train/pooled_df.pkl')
df.head()

,SUBJECT_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,...,diagnosisstring,subject_id,stay_id,first_careunit,last_careunit,intime,outtime,los,anchor_age,dod
0,1027140.0,3011529.0,carevue,CCU,CCU,57.0,57.0,2152-07-23 05:34:12,2152-07-24 18:37:32,1.5440,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
1,1027482.0,3011175.0,carevue,MICU,MICU,12.0,12.0,2162-03-20 17:22:48,2162-03-22 18:26:10,2.0440,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
2,1011746.0,3013234.0,carevue,MICU,MICU,52.0,52.0,2107-07-28 17:22:31,2107-08-01 11:03:44,3.7370,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
3,1010325.0,3022526.0,metavision,MICU,MICU,52.0,52.0,2145-02-06 02:15:08,2145-02-07 18:47:37,1.6892,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT
4,1012708.0,3035542.0,metavision,SICU,SICU,33.0,33.0,2175-02-20 07:41:28,2175-02-23 10:21:01,3.1108,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT


In [20]:
from transformers import AutoTokenizer, AutoModel

# Load the pre-trained model and tokenizer
model_name = "dmis-lab/biobert-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


def embed(input_text):
    tokens = tokenizer(input_text, return_tensors="pt")
    embeddings = model(**tokens).last_hidden_state
    pooled_embeddings = embeddings.mean(dim=1)
    return pooled_embeddings

/Users/noelkj/miniconda3/envs/ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 49.0/49.0 [00:00<00:00, 19.9kB/s]
Downloading: 100%|██████████| 462/462 [00:00<00:00, 162kB/s]
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 2.55MB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 55.8kB/s]
Downloading: 100%|██████████| 433M/433M [00:30<00:00, 14.3MB/s] 


In [21]:
def embed_data(df):
    df_embed = pd.DataFrame()
    for k in range(len(df)):
        for n in range(len(df['code_name'][k])):
            input_text = str(df['code_name'][k][n]) + str(df['value'][k][n]) + str(df['uom'][k][n])
            embed_input = embed(input_text)
            df_embed = df_embed.append({'embed': embed_input}, ignore_index=True)
    return df_embed

# Test with first 10 rows
df_test = df.head(10)
df_test = embed_data(df_test)
df_test.head()

/var/folders/pb/0ltrmd7x4dg5j794jr3tqh1h0000gn/T/ipykernel_35493/1121009204.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_embed = df_embed.append({'embed': embed_input}, ignore_index=True)
